<a href="https://colab.research.google.com/github/gowtham1997/indicTrans-1/blob/patch-1/indictrans_fairseq_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# create a seperate folder to store everything
!mkdir testing
%cd testing

/content/testing


In [ ]:
# clone the repo for running evaluation
!git clone https://github.com/AI4Bharat/indicTrans.git
%cd indicTrans
# clone requirements repositories
!git clone https://github.com/anoopkunchukuttan/indic_nlp_library.git
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git
!git clone https://github.com/rsennrich/subword-nmt.git
%cd ..

Cloning into 'indicTrans'...
remote: Enumerating objects: 366, done.
remote: Counting objects: 100% (366/366), done.
remote: Compressing objects: 100% (237/237), done.
remote: Total 366 (delta 212), reused 251 (delta 125), pack-reused 0
Receiving objects: 100% (366/366), 1.38 MiB | 5.80 MiB/s, done.
Resolving deltas: 100% (212/212), done.
/content/testing/indicTrans
Cloning into 'indic_nlp_library'...
remote: Enumerating objects: 1271, done.
remote: Counting objects: 100% (93/93), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 1271 (delta 50), reused 54 (delta 25), pack-reused 1178
Receiving objects: 100% (1271/1271), 9.56 MiB | 12.94 MiB/s, done.
Resolving deltas: 100% (654/654), done.
Cloning into 'indic_nlp_resources'...
remote: Enumerating objects: 133, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 133 (delta 0), reused 2 (delta 0), pack-reused 126
Receiving objects: 100% (133/133), 149.77 MiB | 

In [ ]:
# Install the necessary libraries
!pip install sacremoses pandas mock sacrebleu tensorboardX pyarrow indic-nlp-library
# Install fairseq from source
!git clone https://github.com/pytorch/fairseq.git
%cd fairseq
# !git checkout da9eaba12d82b9bfc1442f0e2c6fc1b895f4d35d
!pip install --editable ./
%cd ..

     |████████████████████████████████| 901kB 5.2MB/s 
     |████████████████████████████████| 61kB 6.6MB/s 
     |████████████████████████████████| 122kB 14.9MB/s 
Cloning into 'fairseq'...
remote: Enumerating objects: 27747, done.
remote: Counting objects: 100% (168/168), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 27747 (delta 87), reused 117 (delta 75), pack-reused 27579
Receiving objects: 100% (27747/27747), 11.61 MiB | 23.73 MiB/s, done.
Resolving deltas: 100% (20908/20908), done.
/content/testing/fairseq
Obtaining file:///content/testing/fairseq
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 133kB 5.1MB/s 
     |████████████████████████████████| 112kB 8.5MB/s 
     |████████████████████████████████| 645kB 8.5MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-cp

In [ ]:
# download the indictrans model


# downloading the en-indic model
!wget https://akpublicdata.blob.core.windows.net/indicnlp/indictrans/inidctrans-en-indic-v0.2.zip
!unzip inidctrans-en-indic-v0.2.zip

# downloading the indic-en model
!wget https://akpublicdata.blob.core.windows.net/indicnlp/indictrans/indictrans-indic-en-v0.2.zip
!unzip indictrans-indic-en-v0.2.zip

%cd indicTrans/

--2021-04-30 18:33:50--  https://akpublicdata.blob.core.windows.net/indicnlp/indictrans/inidctrans-en-indic-v0.2.zip
Resolving akpublicdata.blob.core.windows.net (akpublicdata.blob.core.windows.net)... 52.239.246.4
Connecting to akpublicdata.blob.core.windows.net (akpublicdata.blob.core.windows.net)|52.239.246.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4609212103 (4.3G) [application/x-zip-compressed]
Saving to: ‘inidctrans-en-indic-v0.2.zip’

inidctrans-en-indic 100%[===================>]   4.29G  22.8MB/s    in 3m 56s  

2021-04-30 18:37:46 (18.6 MB/s) - ‘inidctrans-en-indic-v0.2.zip’ saved [4609212103/4609212103]

unzip:  cannot find or open en-indic.zip, en-indic.zip.zip or en-indic.zip.ZIP.
/content/testing/indicTrans


In [ ]:
# creating a text file and adding en sentences we can use for testing the model
!touch en_sentences.txt
!echo 'This bicycle is too small for you !!' >> en_sentences.txt
!echo "I will directly meet you at the airport." >> en_sentences.txt
!echo 'If COVID-19 is spreading in your community, stay safe by taking some simple precautions, such as physical distancing, wearing a mask, keeping rooms well ventilated, avoiding crowds, cleaning your hands, and coughing into a bent elbow or tissue' >> en_sentences.txt

In [ ]:
# joint_translate takes src_file, output_fname, src_lang, tgt_lang, model_folder as inputs
# src_file -> input text file to be translated
# output_fname -> name of the output file (will get created) containing the model predictions
# src_lang -> source lang code of the input text ( in this case we are using en-indic model and hence src_lang would be 'en')
# tgt_lang -> target lang code of the input text ( tgt lang for en-indic model would be any of the 11 indic langs we trained on:
#              as, bn, hi, gu, kn, ml, mr, or, pa, ta, te)
# supported languages are:
#              as - assamese, bn - bengali, gu - gujarathi, hi - hindi, kn - kannada, 
#              ml - malayalam, mr - marathi, or - oriya, pa - punjabi, ta - tamil, te - telugu

# model_dir -> the directory containing the model and the vocab files

# Note: if the translation is taking a lot of time, please tune the buffer_size and batch_size parameter for fairseq-interactive defined inside this joint_translate script


# here we are translating the english sentences to tamil
!bash joint_translate.sh en_sentences.txt ta_outputs.txt 'en' 'ta' '../en-indic'

Fri Apr 30 20:10:42 UTC 2021
Applying normalization and script conversion
100% 3/3 [00:00<00:00, 50.90it/s]
Number of sentences in input: 3
Applying BPE
Decoding
Extracting translations, script conversion and detokenization
Translation completed


In [ ]:
!cat ta_outputs.txt

இந்த சைக்கிள் உங்களுக்கு மிகவும் சிறியது!
விமான நிலையத்தில் உங்களை நேரில் சந்திக்கிறேன்.
உங்கள் சமூகத்தில் கோவிட்-19 பரவுகிறது என்றால், சில எளிய முன்னெச்சரிக்கை நடவடிக்கைகளான, தனி நபர் இடைவெளி, முகக்கவசம் அணிதல், அறைகளை நன்கு காற்றோட்டமாக வைத்திருத்தல், கூட்டத்தைத் தவிர்த்தல், கைகளைக் கழுவுதல், முழங்கை அல்லது திசுக்களில் இருமல் போன்றவற்றை மேற்கொள்வதன் மூலம் பாதுகாப்பாக இருங்கள்.


In [ ]:
# Similarly, we can translate the english sentences to hindi
!bash joint_translate.sh en_sentences.txt hi_outputs.txt 'en' 'hi' '../en-indic'

Fri Apr 30 20:13:36 UTC 2021
Applying normalization and script conversion
100% 3/3 [00:00<00:00, 53.30it/s]
Number of sentences in input: 3
Applying BPE
Decoding
Extracting translations, script conversion and detokenization
Translation completed


In [ ]:
!cat hi_outputs.txt

यह साइकिल तुम्हारे लिए बहुत छोटी है!
मैं आपसे एयरपोर्ट पर ही मिलने वाला हूं।
यदि आपके समुदाय में कोविड-19 फैल रहा है, तो कुछ सरल सावधानियां बरतें, जैसे शारीरिक दूरी बनाए रखना, मास्क पहनना, कमरों को अच्छी तरह से हवादार रखना, भीड़ से बचना, अपने हाथों को साफ करना और कोहनी या ऊतक को मोड़कर खांसते हुए सुरक्षित रहें


In [ ]:
# creating a text file and adding hi sentences we can use for testing the model
!touch hi_sentences.txt
!echo 'तुम आज सुबह यहाँ क्यों आए?' >> hi_sentences.txt
!echo "मेरे परिवार में हर कोई जल्दी उठता है।" >> hi_sentences.txt
!echo ' स्वास्थ्य और परिवार कल्याण मंत्रालय द्वारा प्रदान की गई जानकारी और सलाह को सावधानी व सही तरीके से पालन कर वायरस के स्थानीय प्रसार को रोका जा सकता है।' >> hi_sentences.txt

!touch ta_sentences.txt
!echo 'அவனுக்கு நம்மைப் தெரியும் என்று தோன்றுகிறது' >> ta_sentences.txt
!echo "இது எங்கே இருக்கு என்று என்னால் கண்டுபிடிக்க முடியவில்லை." >> ta_sentences.txt
!echo 'உங்களுக்கு உங்கள் அருகில் இருக்கும் ஒருவருக்கோ இத்தகைய அறிகுறிகள் தென்பட்டால், வீட்டிலேயே இருப்பது, கொரோனா வைரஸ் தொற்று பிறருக்கு வராமல் தடுக்க உதவும்.' >> ta_sentences.txt

In [ ]:
# here we are translating the english sentences to hindi
!bash joint_translate.sh hi_sentences.txt en_outputs.txt 'hi' 'en' '../indic-en'

Fri Apr 30 20:59:01 UTC 2021
Applying normalization and script conversion
100% 3/3 [00:00<00:00, 58.65it/s]
Number of sentences in input: 3
Applying BPE
Decoding
Extracting translations, script conversion and detokenization
Translation completed


In [ ]:
! cat en_outputs.txt

Why did you come here this morning?
Everyone in my family gets up early.
The local spread of the virus can be curbed by following the information and advice provided by the Ministry of Health and Family Welfare in a careful and correct manner.


In [ ]:
# here we are translating the english sentences to tamil
!bash joint_translate.sh ta_sentences.txt en_outputs.txt 'ta' 'en' '../indic-en'

Fri Apr 30 21:06:20 UTC 2021
Applying normalization and script conversion
100% 3/3 [00:00<00:00, 41.32it/s]
Number of sentences in input: 3
Applying BPE
Decoding
Extracting translations, script conversion and detokenization
Translation completed


In [ ]:
! cat en_outputs.txt

He seems to know us.
I couldnt find it anywhere.
If someone in your neighbourhood develops these symptoms, staying at home can help prevent the spread of the coronavirus infection.


In [ ]:
# to compute bleu scores for the predicitions with a reference file, use the following command

# bash compute_bleu.sh pred_fname ref_fname src_lang tgt_lang
# arguments:
# pred_fname: file that contains model predictions
# ref_fname: file that contains references
# src_lang and tgt_lang : the source and target language